### Notebook to the Project

In this project, I will be using data from https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031/resource/fa64fa70-6328-4d47-bcf0-1eff694d7c22 . However, although more recent data is available, I will be making use of the data from Zurich from the year 2019 - pre-Corona, as the Corona pandemic has biased this situation and I would like our analysis to be more applicable to normal, everyday life.

<img src="geo.png">

In [1]:
#import necessary packages
import pandas as pd
import numpy as np

In [2]:
# Import Traffic Data
traffic = pd.read_csv("sid_dav_verkehrszaehlung_miv_od2031_2019.csv")
traffic.head()

/Users/leander/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,...,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T00:00:00,2020-02-05,201.0,Gemessen
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T01:00:00,2020-02-05,343.0,Gemessen
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T02:00:00,2020-02-05,175.0,Gemessen
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T03:00:00,2020-02-05,99.0,Gemessen
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,...,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T04:00:00,2020-02-05,79.0,Gemessen


In [3]:
#Analyse the headers
Column_Headers = list(traffic.columns.values)
Column_Headers

['MSID',
 'MSName',
 'ZSID',
 'ZSName',
 'Achse',
 'HNr',
 'Hoehe',
 'EKoord',
 'NKoord',
 'Richtung',
 'Knummer',
 'Kname',
 'AnzDetektoren',
 'D1ID',
 'D2ID',
 'D3ID',
 'D4ID',
 'MessungDatZeit',
 'LieferDat',
 'AnzFahrzeuge',
 'AnzFahrzeugeStatus']

In [4]:
#Analyse the data types
traffic.dtypes

MSID                   object
MSName                 object
ZSID                   object
ZSName                 object
Achse                  object
HNr                    object
Hoehe                  object
EKoord                float64
NKoord                float64
Richtung               object
Knummer                 int64
Kname                  object
AnzDetektoren           int64
D1ID                    int64
D2ID                   object
D3ID                   object
D4ID                   object
MessungDatZeit         object
LieferDat              object
AnzFahrzeuge          float64
AnzFahrzeugeStatus     object
dtype: object

In [5]:
#How big is the thing? (It is a gigabyte big...)
traffic.shape

(1579416, 21)

The Column names are in German (official language in Zurich), as a first step next week we will translate them, and clean them up.

We are lucky, as this dataset is pretty comprehensive: It encapsulates the *hourly* traffic density over the period of a whole year!

In [6]:
#Make dataframe out of coordinates
koord = [traffic["EKoord"], traffic["NKoord"]]

head = ["Longitude", "Latitude"]

longlat = pd.concat(koord, axis=1, keys=head)

longlat

,Longitude,Latitude
0,2683009.89,1243936.20
1,2683009.89,1243936.20
2,2683009.89,1243936.20
3,2683009.89,1243936.20
4,2683009.89,1243936.20
...,...,...
1579411,2682704.50,1250615.65
1579412,2682704.50,1250615.65
1579413,2682704.50,1250615.65
1579414,2682704.50,1250615.65


### Here we can see something very annoying: The data set uses the https://de.wikipedia.org/wiki/Schweizer_Landeskoordinaten coordinate system - something used in the alpine expeditions. It is a different projection-type with very different ranges, and we need to look for a package or library that converts our data...

In [18]:
#Make appearances unique

longlatunique = [pd.value_counts(longlat[i]).reset_index().astype(str).apply(' '.join, 1) for i in longlat]
out = pd.concat(longlatunique, 1).fillna('')
out.columns = longlat.columns

longlatunique

[0      2681286.9 17520
 1      2684575.79 8760
 2      2680283.25 8760
 3      2682474.28 8760
 4      2682050.28 8760
             ...       
 176    2681302.84 8760
 177    2684191.84 8760
 178    2687021.11 8760
 179    2685373.47 1728
 180      2685383.0 888
 Length: 181, dtype: object,
 0      1248596.51 17520
 1       1250180.13 8760
 2       1251264.63 8760
 3       1249680.65 8760
 4       1250615.65 8760
              ...       
 176     1248112.09 8760
 177    1247829.965 8760
 178     1247315.24 8760
 179     1251665.11 1728
 180      1251656.05 888
 Length: 181, dtype: object]

In [21]:
import requests
!pip install geopandas
import geopandas
import folium
from shapely.geometry import Point
import os.path
from tqdm.notebook import tqdm

     |████████████████████████████████| 962 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 3.2 MB/s eta 0:00:01


In [59]:
geometry = [Point(xy) for xy in zip(longlat['Longitude'], longlat['Latitude'])]
gdf = geopandas.GeoDataFrame(longlat, geometry=geometry, crs='EPSG:2056')

gdf2 = gdf.to_crs('EPSG:4326')
gdf2
#This should give us the wanted coordinate

,Longitude,Latitude,geometry
0,2683009.89,1243936.20,POINT (8.53707 47.34106)
1,2683009.89,1243936.20,POINT (8.53707 47.34106)
2,2683009.89,1243936.20,POINT (8.53707 47.34106)
3,2683009.89,1243936.20,POINT (8.53707 47.34106)
4,2683009.89,1243936.20,POINT (8.53707 47.34106)
...,...,...,...
1579411,2682704.50,1250615.65,POINT (8.53426 47.40117)
1579412,2682704.50,1250615.65,POINT (8.53426 47.40117)
1579413,2682704.50,1250615.65,POINT (8.53426 47.40117)
1579414,2682704.50,1250615.65,POINT (8.53426 47.40117)


In [ ]:
#Bud sadly, it gives us the conversion as a very weird dataframe format POINT(...)
#We need to convert it to a string and do some magic

In [ ]:
gdf['name'].str.replace(r"\(.*\)","")

In [60]:
gdf2=gdf2.drop(columns=['Longitude'])

In [68]:
gdf2
gdf3=gdf2

gdf3

,Latitude,geometry
0,1243936.20,POINT (8.53707 47.34106)
1,1243936.20,POINT (8.53707 47.34106)
2,1243936.20,POINT (8.53707 47.34106)
3,1243936.20,POINT (8.53707 47.34106)
4,1243936.20,POINT (8.53707 47.34106)
...,...,...
1579411,1250615.65,POINT (8.53426 47.40117)
1579412,1250615.65,POINT (8.53426 47.40117)
1579413,1250615.65,POINT (8.53426 47.40117)
1579414,1250615.65,POINT (8.53426 47.40117)


In [45]:
s = 'POINT (2682704.500 1250615.650)'
start = s.find('(')+1
end = s.find(')', start)
ss=s[start:end]

In [46]:
x = [float(i) for i in ss.split()]

In [47]:
x

[2682704.5, 1250615.65]

In [49]:
gdf.shape

(1579416, 3)

In [65]:
str(gdf2.iloc[0,1])

'POINT (8.537068036893146 47.34105853042033)'

In [74]:
placeholder=[]
for j in range(gdf2.shape[0]):
    a=str(gdf2.iloc[j,1])
    start = a.find('(')+1
    end = a.find(')', start)
    aa=a[start:end]
    placeholder.append(aa)
    
    

In [75]:
placeholder

['8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',
 '8.537068036893146 47.34105853042033',


In [79]:
coordi = pd.DataFrame(placeholder) 
coordi.columns = ['coord']

In [82]:
coordin=coordi.coord.str.split(expand=True,)
coordin.columns=['X','Y']
coordin

,X,Y
0,8.537068036893146,47.34105853042033
1,8.537068036893146,47.34105853042033
2,8.537068036893146,47.34105853042033
3,8.537068036893146,47.34105853042033
4,8.537068036893146,47.34105853042033
...,...,...
1579411,8.534261426307612,47.40116779828502
1579412,8.534261426307612,47.40116779828502
1579413,8.534261426307612,47.40116779828502
1579414,8.534261426307612,47.40116779828502


In [103]:
XX=coordin['X'].unique().tolist()
YY=coordin['Y'].unique().tolist()

d = {'X':XX,'Y':YY}

coordinates = pd.DataFrame(d).astype(float)

In [104]:
coordinates

,X,Y
0,8.537068,47.341059
1,8.537112,47.341071
2,8.529921,47.334167
3,8.529980,47.334165
4,8.517891,47.332547
...,...,...
176,8.551042,47.406761
177,8.596774,47.360546
178,8.596711,47.360518
179,8.534499,47.401226


In [130]:
records = coordinates.to_records(index=False)
coordinatetuples = list(records)
print(coordinatetuples, len(coordinatetuples))

[(8.53706804, 47.34105853), (8.53711183, 47.34107107), (8.52992146, 47.33416671), (8.52997966, 47.33416535), (8.51789117, 47.33254743), (8.51834935, 47.33291231), (8.49462326, 47.36779007), (8.49474456, 47.36768408), (8.48299112, 47.37450361), (8.48296283, 47.37445205), (8.47211014, 47.39399579), (8.47202373, 47.39393224), (8.47362886, 47.39816306), (8.47367362, 47.39823039), (8.48048369, 47.4052424), (8.47907478, 47.40506053), (8.48337573, 47.41595609), (8.48330767, 47.41595193), (8.50443707, 47.41984233), (8.50438686, 47.41979044), (8.54714059, 47.42630738), (8.54646818, 47.42698314), (8.55593217, 47.41995709), (8.55587844, 47.41962587), (8.57993863, 47.40910873), (8.57981753, 47.4090968), (8.59156376, 47.40641562), (8.59142471, 47.40644939), (8.59133762, 47.40563944), (8.5914021, 47.40573522), (8.59217651, 47.39812866), (8.59217489, 47.39816403), (8.57496612, 47.37852611), (8.57544525, 47.37857582), (8.59865098, 47.36083712), (8.59862304, 47.36088561), (8.57555943, 47.35391258), (8.

In [228]:
import folium
latlon = [ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
mapit = folium.Map( location=[47.39314277, 8.492835], zoom_start=11.5 )
for coorda in coordinatetuples:
    folium.Marker( location=[ coorda[1], coorda[0] ]).add_to( mapit )

mapit.save( 'map.html')
mapit

### We have succesfully applied what we learnt in the course, and the locations are now mapped! However, so far we have no idea about the traffic density at these points. We need this, as this is our definite way to make a statement of traffic magnitude in different regions of Zurich.

In [129]:
traffic.Kname.unique().tolist()

['Badanstalt Wollishofen',
 'Albis-/Paradies-/Widmerstr (Schulweg)',
 'Leimbach-/Soodstr',
 'Birmensdorfer-/Triemlistr',
 'Albisriederstr/Lyrenweg',
 'Badenerstr/Hermetschloobrücke/Zürcherstr',
 'Bernerstr/A1 Limmatbrücke',
 'Frankentaler-/Limmattalstr',
 'Frankentaler-/Regensdorferstr',
 'Wehntalerstr 566 (Fg-Uebergang Schulhaus Holderbach)',
 'Zehntenhausplatz',
 'Glatttalstr/Kolbenacker',
 'Schärenmoos-/Thurgauerstr',
 'Ueberlandstr 327 (Weststr. Kanton)',
 'Neue Winterthurer-/Ueberlandstr',
 'Altwiesen-/Dübendorfstr/Hohmoos (Mattenhof)',
 'Dreiwiesen-/Tobelhofstr',
 'Katzenschwanz-/Witikonerstr',
 'Forch-/Witellikerstr',
 'Bellerivestr (Ausfahrt Seepolizei)',
 'A3 Allmendstr',
 'Wallisellenstr/Einfahrt A1 (Stadteinwärts)',
 'Am Wasser/Europabrücke/Winzerstr',
 'Wipkingerplatz (Höngger-/Röschibachstr)',
 'Escher Wyss Platz (Em Sihlquai/Wipkingerbrücke)',
 'Kornhaus-/Rousseaustr',
 'Walcheplatz (Neumühlequai/Walchebrücke/Walchetor)',
 'Bahnhofquai/Bahnhofquai Unterführung Nord',
 'Li

In [131]:
traffic.groupby("Kname")["AnzDetektoren"].count()

Kname
A3 Allmendstr                                           52560
Aargauer-/Pfingstweidstr                                 8760
Albis-/Paradies-/Widmerstr (Schulweg)                   17520
Albisrieder-/Gutstr (Hubertus)                          35040
Albisriederstr/Lyrenweg                                 17520
                                                        ...  
Wallisellenstr/Einfahrt A1 (Stadteinwärts)               2616
Wasserwerkstr (Mibutu Südportal)                        17520
Wehntalerstr 566 (Fg-Uebergang Schulhaus Holderbach)     8760
Wipkingerplatz (Höngger-/Röschibachstr)                 26280
Zehntenhausplatz                                         8760
Name: AnzDetektoren, Length: 88, dtype: int64

In [133]:
traffic.groupby("Kname")["NKoord"]

In [136]:
traffic['X']=coordin['X']
traffic['Y']=coordin['Y']

In [138]:
traffic

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,...,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,X,Y
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T00:00:00,2020-02-05,201.0,Gemessen,8.537068036893146,47.34105853042033
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T01:00:00,2020-02-05,343.0,Gemessen,8.537068036893146,47.34105853042033
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T02:00:00,2020-02-05,175.0,Gemessen,8.537068036893146,47.34105853042033
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T03:00:00,2020-02-05,99.0,Gemessen,8.537068036893146,47.34105853042033
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,...,2,Unbekannt,Unbekannt,Unbekannt,2019-01-01T04:00:00,2020-02-05,79.0,Gemessen,8.537068036893146,47.34105853042033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579411,Z097M002,Unbekannt,Z097,Hofwiesenstrasse (Brunnenhofweg),Hofwiesenstrasse,Unbekannt,ab Brunnenhofweg,2682704.50,1250615.65,Bucheggplatz,...,11,Unbekannt,Unbekannt,Unbekannt,2019-12-31T19:00:00,2020-02-05,326.0,Gemessen,8.534261426307612,47.40116779828502
1579412,Z097M002,Unbekannt,Z097,Hofwiesenstrasse (Brunnenhofweg),Hofwiesenstrasse,Unbekannt,ab Brunnenhofweg,2682704.50,1250615.65,Bucheggplatz,...,11,Unbekannt,Unbekannt,Unbekannt,2019-12-31T20:00:00,2020-02-05,249.0,Gemessen,8.534261426307612,47.40116779828502
1579413,Z097M002,Unbekannt,Z097,Hofwiesenstrasse (Brunnenhofweg),Hofwiesenstrasse,Unbekannt,ab Brunnenhofweg,2682704.50,1250615.65,Bucheggplatz,...,11,Unbekannt,Unbekannt,Unbekannt,2019-12-31T21:00:00,2020-02-05,234.0,Gemessen,8.534261426307612,47.40116779828502
1579414,Z097M002,Unbekannt,Z097,Hofwiesenstrasse (Brunnenhofweg),Hofwiesenstrasse,Unbekannt,ab Brunnenhofweg,2682704.50,1250615.65,Bucheggplatz,...,11,Unbekannt,Unbekannt,Unbekannt,2019-12-31T22:00:00,2020-02-05,317.0,Gemessen,8.534261426307612,47.40116779828502


In [148]:
registry=traffic[['Kname','X','Y']]

In [149]:
registry

,Kname,X,Y
0,Badanstalt Wollishofen,8.537068036893146,47.34105853042033
1,Badanstalt Wollishofen,8.537068036893146,47.34105853042033
2,Badanstalt Wollishofen,8.537068036893146,47.34105853042033
3,Badanstalt Wollishofen,8.537068036893146,47.34105853042033
4,Badanstalt Wollishofen,8.537068036893146,47.34105853042033
...,...,...,...
1579411,Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502
1579412,Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502
1579413,Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502
1579414,Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502


In [203]:
pd.DataFrame(registry.groupby("Kname")["X"].count()).loc[['Badanstalt Wollishofen']]

,X
Kname,
Badanstalt Wollishofen,17520


In [202]:
aaa.iloc[0,0]

17520

In [161]:
len(registry.Kname.unique().tolist())

88

In [162]:
len(registry.X.unique().tolist())

181

In [165]:
registry.Kname.unique().tolist()

['Badanstalt Wollishofen',
 'Albis-/Paradies-/Widmerstr (Schulweg)',
 'Leimbach-/Soodstr',
 'Birmensdorfer-/Triemlistr',
 'Albisriederstr/Lyrenweg',
 'Badenerstr/Hermetschloobrücke/Zürcherstr',
 'Bernerstr/A1 Limmatbrücke',
 'Frankentaler-/Limmattalstr',
 'Frankentaler-/Regensdorferstr',
 'Wehntalerstr 566 (Fg-Uebergang Schulhaus Holderbach)',
 'Zehntenhausplatz',
 'Glatttalstr/Kolbenacker',
 'Schärenmoos-/Thurgauerstr',
 'Ueberlandstr 327 (Weststr. Kanton)',
 'Neue Winterthurer-/Ueberlandstr',
 'Altwiesen-/Dübendorfstr/Hohmoos (Mattenhof)',
 'Dreiwiesen-/Tobelhofstr',
 'Katzenschwanz-/Witikonerstr',
 'Forch-/Witellikerstr',
 'Bellerivestr (Ausfahrt Seepolizei)',
 'A3 Allmendstr',
 'Wallisellenstr/Einfahrt A1 (Stadteinwärts)',
 'Am Wasser/Europabrücke/Winzerstr',
 'Wipkingerplatz (Höngger-/Röschibachstr)',
 'Escher Wyss Platz (Em Sihlquai/Wipkingerbrücke)',
 'Kornhaus-/Rousseaustr',
 'Walcheplatz (Neumühlequai/Walchebrücke/Walchetor)',
 'Bahnhofquai/Bahnhofquai Unterführung Nord',
 'Li

In [167]:
registry2=registry.set_index('Kname', inplace=False)
registry2

,X,Y
Kname,,
Badanstalt Wollishofen,8.537068036893146,47.34105853042033
Badanstalt Wollishofen,8.537068036893146,47.34105853042033
Badanstalt Wollishofen,8.537068036893146,47.34105853042033
Badanstalt Wollishofen,8.537068036893146,47.34105853042033
Badanstalt Wollishofen,8.537068036893146,47.34105853042033
...,...,...
Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502
Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502
Brunnenhofweg/Hofwiesenstr,8.534261426307612,47.40116779828502


In [173]:
print(registry2.loc[['Badanstalt Wollishofen']].iloc[0,0])
print(registry2.loc[['Badanstalt Wollishofen']].iloc[0,1])

8.537068036893146
47.34105853042033


In [204]:
nameu=[]
Xu=[]
Yu=[]
county=[]
for name in registry.Kname.unique().tolist():
    nameu.append(name)
    Xu.append(registry2.loc[[name]].iloc[0,0])
    Yu.append(registry2.loc[[name]].iloc[0,1])
    county.append(pd.DataFrame(registry.groupby("Kname")["X"].count()).loc[[name]].iloc[0,0])
    

print(Xu,Yu,nameu,county)

['8.537068036893146', '8.529921464289464', '8.517891174463774', '8.494623260545982', '8.482991115336937', '8.472110139179229', '8.473628863257241', '8.480483688832571', '8.483375732622852', '8.504437070667967', '8.504386863506099', '8.54714058857696', '8.555932173778455', '8.579938633614873', '8.591563762647848', '8.592176513030354', '8.574966121552066', '8.598650982179922', '8.575559426804348', '8.561824896918994', '8.519877593903185', '8.569938996556107', '8.495214870982634', '8.52359923213398', '8.524226701925894', '8.534802543670775', '8.542352194469021', '8.541644767122648', '8.542737391846446', '8.541929944584098', '8.542753661453927', '8.550536931540316', '8.537517002329833', '8.504469733137643', '8.505073749403644', '8.501528061528308', '8.560474245027706', '8.525735057432788', '8.515117080357291', '8.519518680146433', '8.535484959459794', '8.546537898294932', '8.546007535300971', '8.538706082943783', '8.53534688364897', '8.553433331074576', '8.555966412934342', '8.546355072694

In [181]:
len(Xu)==len(Yu)==len(nameu)

True

In [207]:
MasterDat = pd.DataFrame({
'Name': nameu,
'X': map(float, Xu),
'Y': map(float, Yu),
'Magnitude': map(float, county)
})

In [209]:
MasterData=MasterDat.set_index('Name')
MasterData

,X,Y,Magnitude
Name,,,
Badanstalt Wollishofen,8.537068,47.341059,17520.0
Albis-/Paradies-/Widmerstr (Schulweg),8.529921,47.334167,17520.0
Leimbach-/Soodstr,8.517891,47.332547,17520.0
Birmensdorfer-/Triemlistr,8.494623,47.367790,17520.0
Albisriederstr/Lyrenweg,8.482991,47.374504,17520.0
...,...,...,...
Am Wasser/Hardeggstr (Wechselsignalisation T30),8.503033,47.397238,11664.0
Am Wasser 79,8.503033,47.397238,5856.0
Hardturmstr (Bernoullihäuser(Fg-Uebergang)),8.508304,47.393792,17520.0


In [214]:
for name in MasterData.index:
    print(MasterData.loc[[name]].iloc[0,1])
    

47.34105853042033
47.33416670595233
47.33254743168517
47.3677900720736
47.37450360741726
47.39399578587543
47.39816305979796
47.40524239654592
47.41595608547909
47.41984233052452
47.41979044269867
47.42630738167028
47.41995709179105
47.40910873433025
47.40641562297192
47.39812866392448
47.37852611162638
47.36083711637544
47.3539125847475
47.34899725993962
47.34718931215963
47.41018255900773
47.3992834249068
47.39219831344236
47.39257525016696
47.38686948630757
47.37862239482029
47.37863553823723
47.37439504566227
47.37437775431889
47.36687017183799
47.41425687046129
47.38564492330153
47.40568042500895
47.37259296338495
47.36797428191993
47.37515910301261
47.39489304028644
47.38318543249684
47.37382649394903
47.37586765062965
47.37376281697533
47.37458340324657
47.37458264791513
47.37063091313489
47.3667330382913
47.36915309123307
47.36796655903055
47.37680808858806
47.39312910903473
47.41468070805043
47.39838476925496
47.42962550544531
47.37274909967514
47.35940553384653
47.36705135236

In [227]:
# Make an empty map to start
m = folium.Map(location=[47.374504, 8.482991], zoom_start=11)
 
# One can add markers one by one on the map:
for name in MasterData.index:
   folium.Circle(
      location=[MasterData.loc[[name]].iloc[0,1], MasterData.loc[[name]].iloc[0,0]],
      popup=name,
      radius=MasterData.loc[[name]].iloc[0,2]**1.3*0.0005,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)
 
# Save it as html
m.save('mymap.html')
m

### We did it, we now know precise location data about traffic density.